In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Đọc dữ liệu huấn luyện từ train.csv
train_data = pd.read_csv('data/train.csv')

# Chuyển đổi cột 'date' thành kiểu datetime (nếu chưa làm)
train_data['date'] = pd.to_datetime(train_data['date'])

# Trích xuất các đặc trưng thời gian từ cột 'date'
train_data['hour'] = train_data['date'].dt.hour  # Giờ trong ngày
train_data['day_of_week'] = train_data['date'].dt.dayofweek  # Ngày trong tuần (0 = Monday, 6 = Sunday)

# Sử dụng One-Hot Encoding để mã hóa cột 'period' và 'weekday'
train_data = pd.get_dummies(train_data, columns=['period', 'weekday'], drop_first=True)

# Chia dữ liệu thành X và y
X = train_data[['segment_id', 'hour', 'day_of_week'] + [col for col in train_data.columns if col.startswith('period_') or col.startswith('weekday_')]]  # Các đặc trưng
y = train_data['LOS']  # Nhãn trạng thái (LOS - Level of Service)

# Chia tập dữ liệu thành training và testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập test
y_pred = model.predict(X_test)

# Đánh giá mô hình
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')


Accuracy: 0.5760


In [ ]:
import pandas as pd
import heapq

# Đọc dữ liệu từ các file
nodes = pd.read_csv('data/nodes.csv')
segments = pd.read_csv('data/segments.csv')
segment_status = pd.read_csv('data/segment_status.csv')

# Kiểm tra tên các cột trong segments
print(segments.columns)  # Kiểm tra tên các cột

# Tạo đồ thị từ nodes và segments
graph = {}

# Giả sử các cột đúng là 'from_node' và 'to_node'. Nếu có sự khác biệt, hãy thay đổi tương ứng.
for _, row in segments.iterrows():
    from_node = row['s_node_id']  # Kiểm tra cột này trong file segments.csv
    to_node = row['e_node_id']  # Kiểm tra cột này trong file segments.csv
    if from_node not in graph:
        graph[from_node] = []
    if to_node not in graph:
        graph[to_node] = []
    graph[from_node].append(to_node)
    graph[to_node].append(from_node)

# Hàm tính chi phí cạnh (cost)
def get_edge_cost(from_node, to_node):
    segment_id = segments[(segments['s_node_id'] == from_node) & (segments['e_node_id'] == to_node)]['_id'].values[0]
    status = segment_status[segment_status['segment_id'] == segment_id]['status'].values[0]
    congestion_factor = 1 + 0.5 * status  # Hệ số tắc nghẽn, có thể điều chỉnh
    distance = 10  # Giả sử tất cả các đoạn đường có chiều dài 10, bạn có thể tính toán thực tế
    return distance * congestion_factor

# Hàm Dijkstra tìm đường đi tối ưu
def dijkstra(start, end):
    queue = [(0, start)]  # (chi phí, node)
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous_nodes = {node: None for node in graph}

    while queue:
        current_distance, current_node = heapq.heappop(queue)

        if current_node == end:
            path = []
            while previous_nodes[current_node] is not None:
                path.insert(0, current_node)
                current_node = previous_nodes[current_node]
            return path

        if current_distance > distances[current_node]:
            continue

        for neighbor in graph[current_node]:
            distance = get_edge_cost(current_node, neighbor)
            new_distance = current_distance + distance

            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(queue, (new_distance, neighbor))

    return None  # Không có đường đi

# Tìm đường đi từ điểm A đến điểm B
start_node = 373543511  # ID điểm bắt đầu
end_node = 5738158916  # ID điểm kết thúc

path = dijkstra(start_node, end_node)
print(f'Optimal path: {path}')


Index(['_id', 'created_at', 'updated_at', 's_node_id', 'e_node_id', 'length',
       'street_id', 'max_velocity', 'street_level', 'street_name',
       'street_type'],
      dtype='object')


KeyError: 'status'